<a href="https://colab.research.google.com/github/worldone-web/Team4-Traveler/blob/main/NMF_Model_inpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NMF 모델을 학습하기 위한 라이브러리들!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#라이브러리들을 사용하기 위한 설치작업

In [ ]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162734 sha256=7a0e34158f2951b61049c99642421664c6f1cf1adf0db0fc1ad07bd96e45ac63
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import NMF
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import accuracy


# pandas의 DataFrame으로 전환 후 DataSet 변환

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/G_Project/philadelphia_reviews.csv")

reader = Reader(rating_scale=(1, 5)) # 평점은 1~5 사이로 학습하는 모델
data = Dataset.load_from_df(df[['user_id', 'business_id', 'stars']], reader)

# 훈련세트 80% 테스트세트 20% 나누기

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# NMF 모델 생성 및 학습

In [ ]:
nmf_model = NMF()
nmf_model.fit(trainset)

# 테스트 세트에 대한 예측 및 모델의 성능을 RMSE로 평가

In [ ]:
predictions = nmf_model.test(testset)
accuracy.rmse(predictions)

RMSE: 1.3163


1.3163086789668728

# Precision / Recall로 평가
pred.r_ui는 제공된 DataSet의 실제 평점 <br>
pred.est는 모델이 예측한 평점

In [ ]:
from sklearn.metrics import precision_score, recall_score
import numpy as np

# 예측값과 실제값을 추출
y_true = np.array([int(pred.r_ui >= 3.5) for pred in predictions])
y_pred = np.array([int(pred.est >=3.5) for pred in predictions])

# Precision과 Recall 계산
precision = precision_score(y_true, y_pred )
recall = recall_score(y_true, y_pred )

print("Precision:", precision)
print("Recall:", recall)

Precision: 0.7545029634269744
Recall: 0.7705352046139008


[[TN, FP],[FN, TP]]의 혼합배열을 나타내는 코드이다.
---
혼동 행렬 출력에서는 각 클래스에 해당하는 모든 예측값이 클래스 0에 해당하는 것으로 나타나며, 다른 클래스에 대한 예측값이 없는 것으로 보인다. 0이 아닌 값은 각 클래스를 의미한다.
---





In [ ]:
from sklearn.metrics import confusion_matrix
threshold = 3.5
y_pred_binary = [1 if pred > threshold else 0 for pred in y_pred]
conf_matrix = confusion_matrix(y_true, y_pred_binary)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[    0     0     0     0     0     0]
 [ 6340  8034     0     0     0     0]
 [ 5848  6416     0     0     0     0]
 [ 8470 11024     0     0     0     0]
 [12512 28772     0     0     0     0]
 [10803 49519     0     0     0     0]]


Precision / Recall은 이진 분류 문제에서 사용되는 평가 지표이므로 평점 추천과 같은 다중 클래스 평가 문제에는 직접적으로 적용할 수 없다. Precision과 Recall을 계산하기 위해서는 예측값을 이진 분류로 변환해야 합니다.
위와 같은 평가방법은 잘못된 평가 방법입니다.

In [ ]:
from sklearn.metrics import precision_score, recall_score
import numpy as np

# 예측값과 실제값을 추출
y_true = np.array([pred.r_ui for pred in predictions])
y_pred = np.array([pred.est for pred in predictions])

# Precision과 Recall 계산
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

print("Precision:", precision)
print("Recall:", recall)

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

# 모델의 성능 평가 (MSE 계산)
MSE는 모델이 예측한 값과 실제 값 간의 차이의 제곱을 평균한 값

<center>
<img src="https://drive.google.com/uc?id=1LzmQJD3IMhO42AldpiFBC2cAcht98Yxx" width="500" height="100"><br>
<center/>

In [ ]:
mse = accuracy.mse(predictions)

MSE: 1.7327


# 모델의 성능 평가 (MAE 계산)

모델이 예측한 값과 실제 값 간의 차이의 절댓값을 평균한 값

<center>
<img src="https://drive.google.com/uc?id=1Xnt8D7SD7XNDNxjB0mlwONFmohEUYr1o" height="100"><br>
<center/>

In [ ]:
mae = accuracy.mae(predictions)

MAE:  1.0246


# NMF 모델에 잠재 요인 수와 학습 에포크 수 지정하여 평가

In [ ]:
nmf_model = NMF(n_factors=5, n_epochs=5)
nmf_model.fit(trainset)
predictions = nmf_model.test(testset)

# 모델 학습
accuracy.rmse(predictions)
mse = accuracy.mse(predictions)
mae = accuracy.mae(predictions)

RMSE: 1.6090
MSE: 2.5887
MAE:  1.1624
